# LightRay Analysys

## Author: Fernando kelvin da Silva Soares

In this notebook we will study light reflaction ans refraction, using LightRay analysis and the Snell and Fresnel equations.

In [1]:
# !pip install numpy
# !pip install ipympl
# !pip install matplotlib

In [2]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors, cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

if 'google.colab' in str(get_ipython()):
  %matplotlib inline
else:
  %matplotlib widget

## Create a LightRay, Material and graph class

In [3]:
class LightRay:
    """Defines a LightRay"""
    def __init__(self, r_type:int, angle_deg:int, polarization:str='p', ratio:int=1) -> None: 
        """ Intialize a LightRay vector

            Args:
                r_type: Type of the ray. 0 = Inciddent, 1 = Reflected and 2 = Refracted
                angle: Angle from the normal in degrees
        """
        self.type = r_type
        self.angle = angle_deg
        self.polarization = polarization
        self.ratio = ratio
        self.power = 1
        self.reflectance= None
        self.transmittance= None
        self.layer = None
        self.phase = None
        self.frequency = None
        self.wavelength = None
        self.iteration = None
        self.start_coordinates = None  # (start_x, start_y)
        self.end_coordinates = None    # (end_x, end_y)
        

class Material:
    """Material with properties related to light interaction"""
    
    def __init__(self, name:str,  refractive_index:str, thickness:float) -> None:
        """ Intialize a Material

            Args:
                name: Name of the material
                refractive_index: Index of refraction
                thickness: Material thickness in micro meters
        """
        self.name = name
        self.refractive_index = refractive_index
        self.thickness = thickness


class RayGraph:
    ''' LightRay graph for multiple interfaces'''

    def __init__(self, material_1: Material, material_2: Material, number_of_interfaces:int=1):

        bigger_thickness = material_1.thickness if material_1.thickness > material_2.thickness else material_2.thickness
        hight = bigger_thickness*(number_of_interfaces+2)
        print("hight".format(hight))
        width = bigger_thickness*(number_of_interfaces*2+2)
        self.figure, self.axis = plt.subplots()
        self.axis.set_ylim(-hight, bigger_thickness*2)
        self.axis.set_xlim(-width//2, width//2)
        self.axis.set_ylabel('hight ($\mu$m)')
        self.axis.set_xlabel('width ($\mu$m)')

        interface_y_coord = [0,0] 
        for interface_number in range(number_of_interfaces):
            interface = self.axis.plot([-width//2, width//2], interface_y_coord, 
                                        color='gray', linestyle='dashed', alpha= 0.5, label= 'interface')
            if interface_number//2 == 0:
                add_y = material_1.thickness
            else:
                add_y = material_2.thickness
            interface_y_coord = [interface_y_coord[0]-add_y, interface_y_coord[0]-add_y]
            
        normal = self.axis.plot([0, 0], [-width//2, width//2], 
                                color='yellow', linestyle='dashed', alpha= 0.5, label='normal')

        self.ray_color='blue'
        self.ray_alpha=0.1

    def plot_vector(self, vector: list = [(0, 0),(0, 0)], head_length: float=0.0) -> None:
        """ Plot a vector based on the start and end point
            
            Args: 
                vector: Vector coordinates pair, [(start_x, start_y), (end_x, end_y)].
                head_length: Length of the head of the arrow, that represents the vector.
            """
        a = vector[0]
        b = vector[1]
        dx = b[0] - a[0]
        dy = b[1] - a[1]

        magnitude = sqrt(dx**2+dy**2)

        dx = dx / magnitude
        dy = dy / magnitude

        magnitude = magnitude - head_length

        self.axis.arrow(a[0], a[1], magnitude*dx, magnitude*dy, 
                        head_width=head_length, head_length=head_length, 
                        color=self.ray_color,alpha=self.ray_alpha)
    
    def plot_snell_rays(self, incident_ray: LightRay, material_1: Material, material_2: Material) -> None:
        """Plot the refracted ray for a 2 material interface based on the snell law"""

        incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
        incident_y_start = material_1.thickness
        incident_x_end = 0
        incident_y_end = 0

        self.plot_vector([(incident_x_start,incident_y_start),
                          (incident_x_end,incident_y_end)], 0.1)

        resultant_ray = snell_resultant(incident_ray, material_1, material_2)

        resultant_x_start = 0
        resultant_y_start = 0

        if resultant_ray.type == 1: # Total internal reflection ray
            resultant_x_end = tan(radians(resultant_ray.angle))*material_1.thickness
            resultant_y_end = material_1.thickness
        elif resultant_ray.type == 2: # Refracted ray
            resultant_x_end = tan(radians(resultant_ray.angle))*material_2.thickness
            resultant_y_end = -material_2.thickness
    
        self.plot_vector([(resultant_x_start,resultant_y_start),
                        (resultant_x_end,resultant_y_end)], 0.1)
    
    def plot_fresnel_rays(self, incident_ray: LightRay, material_1: Material, material_2: Material, interface:int=0) -> None:
        """Plot the refracted ray for a 2 material interface based on the snell law"""
        
        offset = interface // 2 * material_1.thickness
        if interface % 2 == 0:    
            offset += interface // 2 * material_2.thickness
            incident_material_thickness = material_1.thickness
        else:
            offset += (interface // 2 + 1) * material_2.thickness
            incident_material_thickness = material_2.thickness
        
        incident_x_start = -tan(radians(incident_ray.angle))*incident_material_thickness
        incident_y_start = -offset + incident_material_thickness
        incident_x_end = 0
        incident_y_end = -offset
        
        self.ray_alpha = 1
        self.plot_vector([(incident_x_start,incident_y_start),
                          (incident_x_end,incident_y_end)], 0.1)

        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)
            
        reflected_x_start = 0
        reflected_y_start = -offset
        reflected_x_end = tan(radians(reflected_ray.angle))*material_1.thickness
        reflected_y_end = -offset + material_1.thickness
        
        self.ray_alpha = reflected_ray.ratio*10 if reflected_ray.ratio*10 <= 1 else 1
        self.plot_vector([(reflected_x_start,reflected_y_start),
                        (reflected_x_end,reflected_y_end)], 0.1)

        transmitted_x_start = 0
        transmitted_y_start = -offset
        transmitted_x_end = tan(radians(transmitted_ray.angle))*material_2.thickness
        transmitted_y_end = -offset -material_2.thickness

        self.ray_alpha = transmitted_ray.ratio*10 if transmitted_ray.ratio*10 <= 1 else 1
        # print("angle: {} R: {} T: {}".format(reflected_ray.angle, reflected_ray.ratio, transmitted_ray.ratio))
        self.plot_vector([(transmitted_x_start,transmitted_y_start),
                        (transmitted_x_end,transmitted_y_end)], 0.1)

    def show(self) -> None:
        plt.show()

## Implement Snell's and Fresnel's equations


In [4]:
def snell_resultant(incident_ray: LightRay, material_1: Material, material_2: Material) -> LightRay:
    """ Compute the refracted ray using the snells law"""

    n1 = material_1.refractive_index # Refractive index of the first material
    n2 = material_2.refractive_index # Refractive index of the second material

    incident_angle = radians(incident_ray.angle)

    # using snell's law to get the transmitted angle
    snell_argument = n1*sin(radians(incident_ray.angle))/n2

    # verify the total internal reflaction condition
    if snell_argument <=  1: 
        transmission_ang = asin(snell_argument)
        resultant_ray = LightRay(2, degrees(transmission_ang))
    else:
        resultant_ray = LightRay(1, degrees(incident_angle))

    return resultant_ray


def fresnel_resultant(incident_ray: LightRay, material_1: Material, material_2: Material) -> LightRay:
    """ Compute the refracted ray using the snells law"""
    n1 = material_1.refractive_index # Refractive index of the first material
    n2 = material_2.refractive_index # Refractive index of the second material
    
    polarization = incident_ray.polarization
    incident_angle = radians(incident_ray.angle)

    # using snell's law to get the transmitted angle
    snell_argument = n1/n2*sin(radians(incident_ray.angle))

    # verify the total internal reflaction condition
    if snell_argument <=  1: 
        transmission_ang = asin(snell_argument)
        if polarization == 's':
            reflectance = ((n1*cos(incident_angle)-n2*cos(transmission_ang))/(n1*cos(incident_angle)+n2*cos(transmission_ang)))**2
        elif polarization == 'p':
            reflectance = ((-n1*cos(transmission_ang)+n2*cos(incident_angle))/(n1*cos(transmission_ang)+n2*cos(incident_angle)))**2
        reflected_ray = LightRay(1, degrees(incident_angle), polarization, reflectance)
        transmitted_ray = LightRay(2, degrees(transmission_ang), polarization, 1-reflectance)
    else:
        transmission_ang = incident_angle
        reflected_ray = LightRay(1, degrees(incident_angle), polarization, 1)
        transmitted_ray = LightRay(2, degrees(transmission_ang), polarization, 0)

    # check the propagation direction of the ray
    if incident_ray.start_coordinates[1] > incident_ray.end_coordinates[1]:
        direction = 1 # downwards direction
    else: 
        direction = -1 # upwards direction

    # calculate the coordinates for the reflected and transmitted rays
    offset_x, offset_y = incident_ray.end_coordinates
    reflected_x_start = offset_x
    reflected_y_start = offset_y
    reflected_x_end = offset_x + tan(radians(reflected_ray.angle))*material_1.thickness
    reflected_y_end = offset_y + direction*material_1.thickness
    
    reflected_ray.start_coordinates = (reflected_x_start, reflected_y_start)
    reflected_ray.end_coordinates = (reflected_x_end, reflected_y_end)

    transmitted_x_start = offset_x
    transmitted_y_start = offset_y
    transmitted_x_end = offset_x + tan(radians(transmitted_ray.angle))*material_2.thickness
    transmitted_y_end = offset_y -direction*material_2.thickness

    transmitted_ray.start_coordinates = (transmitted_x_start, transmitted_y_start)
    transmitted_ray.end_coordinates = (transmitted_x_end, transmitted_y_end)
    
    return reflected_ray, transmitted_ray

## Define a Lightray Exeriment 

Plot the resultant rays for different angles and polarizations based on the snell's and fresnel's laws

In [5]:
def lightray_experiment(mode:str, incident_ray: LightRay, 
                                material_1: Material, material_2: Material, 
                                angle_range:tuple=(0.001, 90),
                                angle_steps:int=30) -> None:
    angles = np.linspace(angle_range[0], angle_range[1], num=angle_steps)

    lightray_graph = RayGraph(material_1, material_2,1)
    lightray_graph.ray_color = [1.0, 0.0, 0.0, 1.0]
    lightray_graph.ray_alpha = 1.0

    incident_ray = LightRay(0, 0.001, polarization='p')
    incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
    incident_y_start = material_1.thickness
    incident_x_end = 0
    incident_y_end = 0

    incident_ray.start_coordinates = (incident_x_start, incident_y_start)
    incident_ray.end_coordinates = (incident_x_end, incident_y_end)

    if mode == 'snell':
        lightray_graph.axis.set_title('Snell analysis - {}/{}'.format(material_1.name,
                                                                        material_2.name))
    elif mode == 'fresnel':
        lightray_graph.axis.set_title('Fresnel analysis - {}/{} \n incident ray polarization = {}'.format(material_1.name,
                                                                                                    material_2.name, 
                                                                                                    incident_ray.polarization))

    for angle in angles:
        if lightray_graph.ray_color[2] + 1/len(angles) < 1:
            lightray_graph.ray_color[2] += 1/len(angles) 
            lightray_graph.ray_color[0] -= 1/len(angles) 
        incident_ray.angle = angle
        if mode == 'snell':
            lightray_graph.plot_snell_rays(incident_ray,material_1,material_2)
        elif mode == 'fresnel':
            lightray_graph.plot_fresnel_rays(incident_ray,material_1,material_2)

    norm = colors.Normalize(vmin=int(angle_range[0]), vmax=int(angle_range[1]))
    cmap = cm.ScalarMappable(norm=norm, cmap=colors.LinearSegmentedColormap.from_list('mycolors',['red','blue']))
    cmap.set_array([])
    lightray_graph.figure.colorbar(cmap, shrink=0.8,label='angle of incidence (degrees)',
                                    ticks=np.linspace(int(angle_range[0]), int(angle_range[1]), num=2))                           

    lightray_graph.show()

## Reflectance Experiment

In [6]:
def reflectance_experiment(material_1: Material, material_2: Material) -> None:
    angles = np.linspace(0.001, 90, num=100)

    fig, ax = plt.subplots()
    ax.set_title('Fresnell reflectance analysis - {}/{}'.format(material_1.name,
                                                                material_2.name))

    ax.set_xlabel("angle (dgrees)")
    ax.set_xlim(0, 90)
    
    p_reflectance = []
    p_transmittance = []
    s_reflectance = []
    s_transmittance = []

    incident_ray = LightRay(0, 0.001, polarization='p')
    incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
    incident_y_start = material_1.thickness
    incident_x_end = 0
    incident_y_end = 0

    incident_ray.start_coordinates = (incident_x_start, incident_y_start)
    incident_ray.end_coordinates = (incident_x_end, incident_y_end)
    current_ray = incident_ray

    for angle in angles:
        incident_ray.angle = angle
        incident_ray.polarization = 'p'
        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)
        p_reflectance.append(reflected_ray.ratio)
        p_transmittance.append(transmitted_ray.ratio)

        incident_ray.polarization = 's'
        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)
        s_reflectance.append(reflected_ray.ratio)
        s_transmittance.append(transmitted_ray.ratio)

    ax.plot(angles, p_reflectance, 'b', label='Rp')
    ax.plot(angles, p_transmittance, 'r', label='Tp')
    ax.plot(angles, s_reflectance, 'b--', label='Rs')
    ax.plot(angles, s_transmittance, 'r--', label='Ts')
    
    plt.legend()
    plt.show()

### Cration of an experiment to se transmission power decay in periodic structures



In [7]:
def periodic_experiment(incident_ray: LightRay, material_1: Material,
                         material_2: Material, number_of_interfaces:int) -> None:
                         
    angles = np.linspace(0.001, 90, num=100)

    incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
    incident_y_start = material_1.thickness
    incident_x_end = 0
    incident_y_end = 0

    incident_ray.start_coordinates = (incident_x_start, incident_y_start)
    incident_ray.end_coordinates = (incident_x_end, incident_y_end)
    current_ray = incident_ray

    current_ray = incident_ray
    ray_power = [1]
    for interface in range(number_of_interfaces):
        if interface % 2 == 1: # Check if it is an odd interface
            reflected_ray, transmitted_ray = fresnel_resultant(current_ray, material_1, material_2)
        else:
            reflected_ray, transmitted_ray = fresnel_resultant(current_ray, material_2, material_1)
        current_ray = transmitted_ray
        ray_power.append(ray_power[-1]*transmitted_ray.ratio)

    fig, ax = plt.subplots()
    ax.set_title('Power transmission trught {} interfaces - {}/{}\n'.format(number_of_interfaces,
                                                                                material_1.name,
                                                                                material_2.name))

    ax.set_xlabel("interfaces")
    ax.set_xlim(0, number_of_interfaces)
    ax.set_ylabel("relative power")
    plt.yticks(np.arange(-0.1, 1.1, 0.1))
    ax.set_ylim(-0.1, 1.1)

    ax.plot(ray_power, 'r', label='{}$^\circ$'.format(incident_ray.angle))
    ax.axhline(y=ray_power[-1], color='black', linestyle='--')
    
    plt.legend()
    plt.show()



### Plot the refracted ray using the snell's and fresnel's law for vacuum and water for different angles and polarizations

In [8]:
plt.close('all')

material_1 = Material('vacuum', 1.0, 5.0)
material_2 = Material('water', 1.33, 5.0)

reflectance_experiment(material_1, material_2)


incident_ray = LightRay(0, 0.001, polarization='p')
lightray_experiment('snell',incident_ray, material_1, material_2)
lightray_experiment('fresnel',incident_ray, material_1, material_2)

incident_ray = LightRay(0, 0.001, polarization='s')
lightray_experiment('fresnel',incident_ray, material_1, material_2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hight


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hight


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hight


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot the refracted ray using the snell's and fresnel's law for optoc fiber and fbg for different angles and polarizations

In [9]:
material_1 = Material('optic_fiber', 1.45, 5)
material_2 = Material('fbg', 1.4005, 5.0)

# reflectance_experiment(material_1, material_2)

incident_ray = LightRay(0, 0.001, polarization='p')
lightray_experiment('snell',incident_ray, material_1, material_2)
lightray_experiment('fresnel',incident_ray, material_1, material_2)

incident_ray = LightRay(0, 0.001, polarization='s')
lightray_experiment('fresnel',incident_ray, material_1, material_2)

hight


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hight


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

hight


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Periodic experiment to see intensity decay for multiple interfaces

In [10]:
def periodic_experiment_multiple_angles( material_1: Material,
                                        material_2: Material, 
                                        number_of_interfaces:int,
                                        polarization:str='p',
                                        angle_range:tuple=(0.001,90),
                                        steps: int=10) -> None:
    fig, ax = plt.subplots()
    ray_color = [1.0, 0.0, 0.0, 1.0]

    angles = np.linspace(angle_range[0], angle_range[1], num=steps)
    
    for angle in angles:
        incident_ray = LightRay(0, angle, polarization=polarization)
        incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
        incident_y_start = material_1.thickness
        incident_x_end = 0
        incident_y_end = 0

        incident_ray.start_coordinates = (incident_x_start, incident_y_start)
        incident_ray.end_coordinates = (incident_x_end, incident_y_end)

        ray_power = [1]

        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)

        for interface in range(number_of_interfaces):
            ray_power.append(ray_power[-1]*transmitted_ray.ratio)

        if ray_color[2] + 1/len(angles) < 1:
            ray_color[2] += 1/len(angles) 
            ray_color[0] -= 1/len(angles) 
        ax.plot(ray_power,label='{}$^\circ$'.format(int(angle)), color=colors.to_rgba_array(ray_color))


    ax.set_title('Power transmission trught {} interfaces - {}/{}\npolarization= {}'.format(number_of_interfaces,
                                                                                            material_1.name,
                                                                                            material_2.name,
                                                                                            polarization))
    ax.set_xlabel("interfaces")
    ax.set_xlim(0, number_of_interfaces)
    ax.set_ylabel("relative power")
    plt.yticks(np.arange(-0.1, 1.1, 0.1))
    ax.set_ylim(-0.1, 1.1)


    norm = colors.Normalize(vmin=int(angle_range[0]), vmax=int(angle_range[1]))
    cmap = cm.ScalarMappable(norm=norm, cmap=colors.LinearSegmentedColormap.from_list('mycolors',['red','blue']))
    cmap.set_array([])
    fig.colorbar(cmap, shrink=0.8,label='angle of incidence (degrees)',
                                    ticks=np.linspace(int(angle_range[0]), int(angle_range[1]), num=2))   
    # plt.legend()
    plt.show()

In [11]:
material_1 = Material('optic_fiber', 1.45, 5)
material_2 = Material('fbg', 1.4, 5)

periodic_experiment_multiple_angles(material_1, material_2, number_of_interfaces=10000, polarization='p', steps=20)
periodic_experiment_multiple_angles(material_1, material_2, number_of_interfaces=10000, polarization='s', steps=20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Transmission power at each layer, changing the refractive index contrast

In [12]:
def periodic_experiment_multiple_contrasts( incident_ray: LightRay,
                                            material_1: Material,
                                            material_2: Material, 
                                            number_of_interfaces:int,
                                            contrasts_list:list) -> None:
    fig, ax = plt.subplots()

    for contrast in contrasts_list:

        current_ray = incident_ray
        material_1.refractive_index = contrast[0]
        material_2.refractive_index = contrast[1]
        
        ray_power = [1]
        
        reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)

        for interface in range(number_of_interfaces):
            ray_power.append(ray_power[-1]*transmitted_ray.ratio)

        ax.plot(ray_power,label='{}/{}'.format(contrast[0],contrast[1]))


    ax.set_title('Power transmission trught {} interfaces - {}/{}\npolarization= {}'.format(number_of_interfaces,
                                                                                            material_1.name,
                                                                                            material_2.name,
                                                                                            incident_ray.polarization))
    ax.set_xlabel("interfaces")
    ax.set_xlim(0, number_of_interfaces)
    ax.set_ylabel("relative power")
    plt.yticks(np.arange(-0.1, 1.1, 0.1))
    ax.set_ylim(-0.1, 1.1)

    plt.legend(title='contrast')
    plt.show()

In [13]:
material_1 = Material('optic_fiber', 1.45, 5)
material_2 = Material('fbg', 1.4, 5)

incident_ray.layer = 0
incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
incident_y_start = material_1.thickness
incident_x_end = 0
incident_y_end = 0
incident_ray.start_coordinates = (incident_x_start, incident_y_start)
incident_ray.end_coordinates = (incident_x_end, incident_y_end)

contrasts_list = [(1.45,1.4), (1.45,1.35), (1.45,1.3), (1.45,1.25), (1.45,1.2)]
periodic_experiment_multiple_contrasts( incident_ray, material_1,material_2, 10000, contrasts_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Ressonance Experiemt

In [14]:
def resonance_experiment_multiple_wavelengths(incident_ray: LightRay, 
                                                material_1: Material, material_2: Material, 
                                                wavelength_range:tuple=(1539, 1540.5),
                                                wavelength_steps:int=200,
                                                number_of_interfaces:list=[1]) -> None:
    

    wavelengths = np.linspace(wavelength_range[0], wavelength_range[1], num=wavelength_steps)

    

    reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)
    print("reflectance: {} | transmittance: {}".format(reflected_ray.ratio, transmitted_ray.ratio))

    fig, ax = plt.subplots()
    ax.set_title('Reflected sum for - {}/{}\n'.format(material_1.name, material_2.name))
    ax.set_xlabel("wavelength")
    ax.set_ylabel("relative power")


    for number in number_of_interfaces:
        reflected_sum_list = []
        for wavelength in wavelengths:

            incident_ray.wavelength = wavelength
            wavelength_material_1 = incident_ray.wavelength/material_1.refractive_index
            wavelength_material_2 = incident_ray.wavelength/material_2.refractive_index

            path_1 = (material_1.thickness*1000)/cos(radians(reflected_ray.angle))
            path_2 = (material_2.thickness*1000)/cos(radians(transmitted_ray.angle))

            phase_shift_1 = ((path_1%wavelength_material_1)*2*pi)/wavelength_material_1
            phase_shift_2 = ((path_2%wavelength_material_2)*2*pi)/wavelength_material_2

            reflected_sum = 0
            for interface in range(number):
                if interface % 2 == 0:
                    total_phase_shift = interface*phase_shift_1+interface*phase_shift_2
                else:
                    total_phase_shift = (interface-1)*phase_shift_1+(interface+1)*phase_shift_2
                amplitude = reflected_ray.ratio*(transmitted_ray.ratio**2)**interface
                reflected_sum = reflected_sum + amplitude*cos(total_phase_shift)
            reflected_sum_list.append(reflected_sum**2)

        ax.plot(wavelengths, reflected_sum_list, label=number)

    plt.legend()
    plt.show()                     


In [15]:
material_1 = Material('optic_fiber', 1.45, 1)
material_2 = Material('fbg', 1.4505, 1)

incident_ray = LightRay(0, 0.01, polarization='s')

incident_ray.layer = 0
incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
incident_y_start = material_1.thickness
incident_x_end = 0
incident_y_end = 0
incident_ray.start_coordinates = (incident_x_start, incident_y_start)
incident_ray.end_coordinates = (incident_x_end, incident_y_end)

resonance_experiment_multiple_wavelengths(incident_ray,material_1, material_2,wavelength_range= (500, 1000), wavelength_steps=50000, number_of_interfaces=[500])

reflectance: 2.97162699897167e-08 | transmittance: 0.99999997028373


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
material_1 = Material('optic_fiber', 1.45, 5)
material_2 = Material('fbg', 1.4505, 5)

incident_ray = LightRay(0, 0.01, polarization='s')

incident_ray.layer = 0
incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
incident_y_start = material_1.thickness
incident_x_end = 0
incident_y_end = 0
incident_ray.start_coordinates = (incident_x_start, incident_y_start)
incident_ray.end_coordinates = (incident_x_end, incident_y_end)

resonance_experiment_multiple_wavelengths(incident_ray,material_1, material_2,wavelength_range= (966, 968), wavelength_steps=10000, number_of_interfaces=[100,200,300])

reflectance: 2.97162699897167e-08 | transmittance: 0.99999997028373


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### For multiple angles

In [17]:
def resonance_experiment_multiple_angles(incident_ray: LightRay, 
                                            material_1: Material, material_2: Material, 
                                            angle_range:tuple=(0.001, 80),
                                            angle_steps:int=100,
                                            number_of_interfaces:list=[1]) -> None:
    
    wavelength_material_1 = incident_ray.wavelength/material_1.refractive_index
    wavelength_material_2 = incident_ray.wavelength/material_2.refractive_index

    angles = np.linspace(angle_range[0], angle_range[1], num=angle_steps)


    fig, ax = plt.subplots()
    ax.set_title('Reflected sum for {} interfaces - {}/{}\n'.format(number_of_interfaces,
                                                                                material_1.name,
                                                                                material_2.name))

    for number in number_of_interfaces:
        reflected_sum_list = []
        for angle in angles:
            
            incident_ray.angle = angle
            reflected_ray, transmitted_ray = fresnel_resultant(incident_ray, material_1, material_2)

            path_1 = (material_1.thickness*1000)/cos(radians(reflected_ray.angle))
            path_2 = (material_2.thickness*1000)/cos(radians(transmitted_ray.angle))

            phase_shift_1 = ((path_1%wavelength_material_1)*2*pi)/wavelength_material_1
            phase_shift_2 = ((path_2%wavelength_material_2)*2*pi)/wavelength_material_2

            reflected_sum = 0
            for interface in range(number):
                if interface % 2 == 0:
                    total_phase_shift = interface*phase_shift_1+interface*phase_shift_2
                else:
                    total_phase_shift = (interface-1)*phase_shift_1+(interface+1)*phase_shift_2
                amplitude = reflected_ray.ratio*(transmitted_ray.ratio**2)**interface
                reflected_sum = reflected_sum + amplitude*cos(total_phase_shift)
        
            reflected_sum_list.append(reflected_sum**2)
        ax.plot(angles,reflected_sum_list, label=number)
    
    ax.set_xlabel("angle")
    ax.set_ylabel("relative power")

    plt.legend()
    plt.show()                     

In [18]:
material_1 = Material('optic_fiber', 1.45, 5)
material_2 = Material('fbg', 1.4505, 5)

incident_ray = LightRay(0, 0.01, polarization='s')

incident_ray.layer = 0
incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
incident_y_start = material_1.thickness
incident_x_end = 0
incident_y_end = 0
incident_ray.start_coordinates = (incident_x_start, incident_y_start)
incident_ray.end_coordinates = (incident_x_end, incident_y_end)

incident_ray.wavelength = 1550

resonance_experiment_multiple_angles(incident_ray,material_1, material_2,angle_range= (0, 20), angle_steps=1000, number_of_interfaces=[500, 10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Periodic structure experiemnt

In [19]:
def periodic_structure_experiment(incident_ray: LightRay, 
                                material_1: Material, material_2: Material, 
                                number_of_interfaces:int=1) -> None:
    lightray_graph = RayGraph(material_1, material_2,number_of_interfaces)
    lightray_graph.ray_color = [1.0, 0.0, 0.0, 1.0]
    lightray_graph.ray_alpha = 1.0

    lightray_graph.axis.set_title('Fresnel analysis - {}/{} \n incident ray polarization = {}'.format(material_1.name,
                                                                                                material_2.name, 
                                                                                                incident_ray.polarization))
    lightray_graph.plot_vector([incident_ray.start_coordinates, incident_ray.end_coordinates])

    incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
    incident_y_start = material_1.thickness
    incident_x_end = 0
    incident_y_end = 0

    incident_ray.start_coordinates = (incident_x_start, incident_y_start)
    incident_ray.end_coordinates = (incident_x_end, incident_y_end)
    current_ray = incident_ray
    
    for interface_number in range(0, number_of_interfaces):
        if interface_number % 2 == 0:
            reflected_ray, transmitted_ray = fresnel_resultant(current_ray, material_1, material_2)
        else:
            reflected_ray, transmitted_ray = fresnel_resultant(current_ray, material_2, material_1)
        lightray_graph.plot_vector([reflected_ray.start_coordinates, reflected_ray.end_coordinates])
        lightray_graph.plot_vector([transmitted_ray.start_coordinates, transmitted_ray.end_coordinates])
        current_ray = transmitted_ray
        

    lightray_graph.show()
    
material_1 = Material('vacuum', 1.0, 5.0)
material_2 = Material('water', 1.33, 5.0)


incident_ray = LightRay(0, 10, polarization='s')
incident_ray.layer = 0

incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
incident_y_start = material_1.thickness
incident_x_end = 0
incident_y_end = 0

incident_ray.start_coordinates = (incident_x_start, incident_y_start)
incident_ray.end_coordinates = (incident_x_end, incident_y_end)

# periodic_structure_experiment(incident_ray, material_1, material_2,50)